# World Generation Script

**Imports**

In [36]:
import google.generativeai as genai
from app_config import LLM_API_KEY,LLM_MODEL
import json
from datetime import datetime

**Prompts**

In [ ]:
system_prompt = f"""
    Seu trabalho é ajudar a criar mundos de fantasia interessantes que os jogadores adorariam explorar.
   
    Instruções:
    Gere apenas texto simples, sem formatação.
    Use uma linguagem simples e clara, sem floreios.
    Cada descrição deve ter no máximo 3-5 frases.
    O conteúdo deve ser gerado em português
"""


world_prompt = f"""
Gere uma descrição criativa para um mundo de fantasia único.

Saída do conteúdo no formato:
Nome do Mundo: <NOME DO MUNDO>
Descrição do Mundo: <DESCRIÇÃO DO MUNDO>

Nome do Mundo:"""

SCHEMA_KINGDOMS={"type":"object","properties":{"kingdoms":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"}},"required":["name","description","world"]}}},"required":["kingdoms"]}

SCHEMA_WORLD={"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"}}}

SCHEMA_TOWNS={"type":"object","properties":{"towns":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"},"kingdom":{"type":"string"}},"required":["name","description","world","kingdom"]}}},"required":["towns"]}

SCHEMA_NPCS={"type":"object","properties":{"npcs":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"},"kingdom":{"type":"string"},"town":{"type":"string"}},"required":["name","description","world","kingdom","town"]}}},"required":["npcs"]}



**Model Setup**

In [ ]:

def generate_config(schema):
    return{
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 8192,
        "response_mime_type": "application/json",
        "response_schema": schema
        
}

genai.configure(api_key=LLM_API_KEY)

client = genai.GenerativeModel(
            model_name=LLM_MODEL,
            system_instruction=system_prompt
            )


## Generating a World

In [ ]:
output = client.generate_content(world_prompt,generation_config=generate_config(SCHEMA_WORLD))

world =json.loads(output.text)
print(world)

## Generating Kingdoms

In [8]:
kingdom_prompt = f"""
Crie 3 reinos diferentes para um mundo de fantasia.
Para cada reino, gere uma descrição com base no mundo em que ele está.
Descreva líderes importantes, culturas e história do reino.

Saída do conteúdo no formato:
Nome do Reino 1: <NOME DO REINO>
Descrição do Reino 1: <DESCRIÇÃO DO REINO>
Nome do Reino 2: <NOME DO REINO>
Descrição do Reino 2: <DESCRIÇÃO DO REINO>
Nome do Reino 3: <NOME DO REINO>
Descrição do Reino 3: <DESCRIÇÃO DO REINO>

Nome do Mundo: {world['name']}
Descrição do Mundo: {world['description']}

Reino 1"""


In [ ]:
output = client.generate_content(kingdom_prompt,generation_config=generate_config(SCHEMA_KINGDOMS))
kingdoms = json.loads(output.text)["kingdoms"]

## Generating Towns

In [10]:
def get_town_prompt(world, kingdom):
    return f"""
            Crie 3 cidades diferentes para um reino e mundo de fantasia.
            Descreva a região em que está localizada, lugares importantes da cidade
            e sua história interessante.

            Saída do conteúdo no formato:
            Nome da Cidade 1: <NOME DA CIDADE>
            Descrição da Cidade 1: <DESCRIÇÃO DA CIDADE>
            Nome da Cidade 2: <NOME DA CIDADE>
            Descrição da Cidade 2: <DESCRIÇÃO DA CIDADE>
            Nome da Cidade 3: <NOME DA CIDADE>
            Descrição da Cidade 3: <DESCRIÇÃO DA CIDADE>

            Nome do Mundo: {world['name']}
            Descrição do Mundo: {world['description']}

            Nome do Reino: {kingdom['name']}
            Descrição do Reino: {kingdom['description']}

            Nome da Cidade 1:"""


In [11]:
def create_towns(world, kingdom):
    print(kingdom['name'])
    output = client.generate_content(get_town_prompt(world,kingdom),generation_config=generate_config(SCHEMA_TOWNS))
    towns = json.loads(output.text)
    kingdom["towns"] = towns["towns"]

In [12]:
for kingdom in kingdoms:
    create_towns(world, kingdom)  

Aethelgard
Silvanus
Mystwood


In [13]:
kingdoms[0]['towns'][0]

{'description': 'Lumina é uma cidade costeira, construída sobre penhascos rochosos e banhada pela luz solar constante. Seu porto movimentado é um centro de comércio e transporte, com navios vindos de todos os cantos de Solara.  O Farol de Cristal, um marco da cidade, guia os navios à noite e protege-os das tempestades. Seu maior evento anual é o Festival da Colheita, que celebra os abundantes frutos do mar e os campos cultivados.',
 'kingdom': 'Aethelgard',
 'name': 'Lumína',
 'world': 'Aetheria'}

In [14]:
town = kingdoms[0]['towns'][0]
print(f'\nTown 1 Description: \
{town["description"]}')


Town 1 Description: Lumina é uma cidade costeira, construída sobre penhascos rochosos e banhada pela luz solar constante. Seu porto movimentado é um centro de comércio e transporte, com navios vindos de todos os cantos de Solara.  O Farol de Cristal, um marco da cidade, guia os navios à noite e protege-os das tempestades. Seu maior evento anual é o Festival da Colheita, que celebra os abundantes frutos do mar e os campos cultivados.


## Generating Non-Player Characters (NPC's)

In [16]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
            Crie 3 personagens diferentes com base no mundo, reino e cidade em que estão. Descreva a aparência e a profissão do personagem, bem como suas dores e desejos mais profundos.

            Saída do conteúdo no formato:
            Nome do Personagem 1: <NOME DO PERSONAGEM>
            Descrição do Personagem 1: <DESCRIÇÃO DO PERSONAGEM>
            Nome do Personagem 2: <NOME DO PERSONAGEM>
            Descrição do Personagem 2: <DESCRIÇÃO DO PERSONAGEM>
            Nome do Personagem 3: <NOME DO PERSONAGEM>
            Descrição do Personagem 3: <DESCRIÇÃO DO PERSONAGEM>

            Nome do Mundo: {world['name']}
            Descrição do Mundo: {world['description']}

            Nome do Reino: {kingdom['name']}
            Descrição do Reino: {kingdom['description']}

            Nome da Cidade: {town['name']}
            Descrição da Cidade: {town['description']}
            
            Nome do Personagem 1:"""

In [17]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.generate_content(get_npc_prompt(world, kingdom, town),generation_config=generate_config(SCHEMA_NPCS))
    npcs = json.loads(output.text)
    town["npcs"] = npcs['npcs']

In [18]:
for kingdom in kingdoms:
    for town in kingdom['towns']:
        create_npcs(world, kingdom, town)



Creating characters for the town of: Lumína...

Creating characters for the town of: Aethelgard...

Creating characters for the town of: Solis...

Creating characters for the town of: Lumina...

Creating characters for the town of: Arboria...

Creating characters for the town of: Solis...

Creating characters for the town of: Elderwood...

Creating characters for the town of: Oakhaven...

Creating characters for the town of: Whisperwind...


In [22]:
npc = town['npcs'][0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Whisperwind, Mystwood:
Elara é uma jovem mulher de cabelos longos e negros como a noite, olhos verdes brilhantes e pele clara. Ela é uma curandeira habilidosa, conhecida por seu toque suave e sua capacidade de acalmar até mesmo as criaturas mais selvagens. Sua dor mais profunda é a perda de seus pais, assassinados durante um ataque a Whisperwind. Seu desejo é proteger seu povo e manter a paz em sua cidade.


## Save the World

In [ ]:
world["kingdoms"]=kingdoms
def save_world(world, filename):
    with open(filename, 'w', encoding='utf8') as f:
        json.dump(world,f, indent=4, ensure_ascii=False)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
def generate_file_name():
       today = str(datetime.now().strftime("%Y-%m-%dT%H%M%SZ"))
       file_name=f"mundos_criados//{today}_mundo_criado.json"
       return file_name        

save_world(world,generate_file_name())